# Reactive Programming Tutorial
### With elements from functional programming

In [1]:
import reactivex as rx
import reactivex.operators as op

In [17]:
names = rx.of("Albert", "Edna", "Gertrude", "Hannah", "Elle", "Anna", "Booty", "Samantha", "Cornelia", "Sherman")
prohibited_names = {"Booty"}

In [10]:
"""Functional Style method to identify palindromes"""
def isPalindrome(s):

    if len(s) <= 1:
        return True
    
    new_s = s.lower()
    
    if (new_s[0] == new_s[-1]):
        return isPalindrome(new_s[1:len(new_s)-1])
    
    else:
        return False

In [18]:
player_stream = names.pipe(
    op.filter(lambda n : (n not in prohibited_names) and (not isPalindrome(n)))
)
# After filtering, player_stream should have 6 items

received Albert
received Edna
received Gertrude
received Samantha
received Cornelia
received Sherman


Playing the game

In [19]:
r1_points = rx.of(0, 10, 3, 4, 12, 6)

# NOTE: WE MIGHT BE ABLE TO ASSIGN THESE TEAMS BASED ON THE PLAYERS STREAM, AND THAT WOULD BE A BETTER WAY OF DOING THINGS, 
# BUT I'M HUNGRY AND TIRED, SO I'M NOT GOING TO TRY TO FIGURE THAT OUT RIGHT NOW
red_team = {"Albert", "Gertrude", "Cornelia"}
blue_team = {"Edna", "Samantha", "Sherman"}

In [ ]:
scores = op.zip(player_stream, r1_points)   # Creates a stream of tuples in the form (player_name, score)

r1_point_threshold = 0     # We can change this if we want people to be filtered out based on their score

# Then the idea is to use the zipped streams that matches the players to the scores in order to assign point values to each team
# Each team should have a "point" stream that we somehow append scores too. IDK if this is actually a valid thing to do, I was
# just trying to incorporate more streams, but it would really just make more sense to update a global "team score" variable

### Final Round

In [ ]:
# Uses the filtered list of players who are still remaining
# Create a new stream of point values
# This time we'll be doing some transformations on the point values
    # if the score is divisible by 2, The score is doubled
    # if the score is divisible by 3, the score is divided by 3 :(

# Same process as above for assigning point values to each team

# PROCESSING THE FINAL RESULTS (if scores are appended to team point stream)
# Go through each team's point stream
# Update global point scores
# Using the point stream's on_complete method, print out the final score for each team